# **⚠️ ATUR RUNTIME TERLEBIH DAHULU**

Runtime perlu disesuaikan dulu untuk menggunakan MediaPipe Model Maker di Google Colab.

Ini karena beberapa dependencies untuk MediaPipe Model Maker tidak kompatibel dengan Python 3.12 keatas.

---

**Di menu atas, buka: Runtime > Ubah jenis runtime > Versi Runtime > 2025.07**

Anda juga bisa mengubah akselerator hardware ke GPU atau TPU jika ingin mempercepat proses training nanti.

**Install prerequisities**


In [ ]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

**Impor modul-modul**


In [ ]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

**Ekstrak zip dataset (upload dulu di file)**

In [ ]:
!unzip finger_count_dataset.zip
dataset_path = "finger_count_dataset"

**Verifikasi manual folder-folder dataset**

In [ ]:
print(dataset_path)
labels = []
for i in os.listdir(dataset_path):
  if os.path.isdir(os.path.join(dataset_path, i)):
    labels.append(i)
print(labels)

**Load dataset**

80% data akan digunakan untuk training, 10% untuk validasi dan 10% lagi untuk testing.

In [ ]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

**Mulai training model**

In [ ]:
hparams = gesture_recognizer.HParams(export_dir="exported_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

**Evaluasi performa model**

In [ ]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

**Download model**

Masukkan filename jika ingin

In [ ]:
filename = "finger_number_recognizer" # @param {type:"string"}

model.export_model()
!cp exported_model/gesture_recognizer.task exported_model/{filename}.task
files.download(f'exported_model/{filename}.task')